# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 임포트

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob

# Darknet 다운받아서 컴파일

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15521, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15521 (delta 0), reused 5 (delta 0), pack-reused 15514
Receiving objects: 100% (15521/15521), 14.19 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


In [ ]:
%cd darknet

/content/darknet


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile

In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |             ^~
./src/image_openc

# 데이터 준비
- darknet/data/obj에 압축 풀기
- classes.names와 training.data 파일 생성
- training.txt 파일 생성

In [ ]:
!cp /content/Cars.zip ./

cp: cannot stat '/content/Cars.zip': No such file or directory


In [ ]:
!rm -rf data/obj
!mkdir data/obj
!unzip Cars.zip -d data/obj

Archive:  Raccoon.zip
   creating: data/obj/Raccoon/
   creating: data/obj/Raccoon/valid/
  inflating: data/obj/Raccoon/valid/raccoon-95_jpg.rf.f8c8616fc565907a66c86128839a766b.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-82_jpg.rf.00ff88031083cd95bc62294500428bb0.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-90_jpg.rf.50b2f1ed0bc307e1e94817b5e2bb361b.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-79_jpg.rf.f133cb7a619420f00681d1326ac28a1b.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-80_jpg.rf.72b3c7ca78af0a414822f9bedfce3b76.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-88_jpg.rf.c3f2a39f336897da05cb3acb9d42ec54.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-73_jpg.rf.8b19ae0bb7646bcaac6f8df04aee44d6.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-78_jpg.rf.5436f171ab5a7b4fbcdad1946a086374.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-86_jpg.rf.a3a11438be66477d5bbab57d14f521d9.jpg  
  inflating: data/obj/Raccoon/valid/raccoon-83_jpg.rf.9111bbd932a9eb3c0eb

### 학습 위한 설정 파일 생성

#### data/obj.names

In [ ]:
%%writefile data/obj.names
chevrolet_malibu_sedan_2012_2016
chevrolet_malibu_sedan_2017_2019
chevrolet_spark_hatchback_2016_2021
chevrolet_trailblazer_suv_2021_
chevrolet_trax_suv_2017_2019
genesis_g80_sedan_2016_2020
genesis_g80_sedan_2021_
genesis_gv80_suv_2020_
hyundai_avante_sedan_2011_2015
hyundai_avante_sedan_2020_
hyundai_grandeur_sedan_2011_2016
hyundai_grandstarex_van_2018_2020
hyundai_ioniq_hatchback_2016_2019
hyundai_sonata_sedan_2004_2009
hyundai_sonata_sedan_2010_2014
hyundai_sonata_sedan_2019_2020
kia_carnival_van_2015_2020
kia_carnival_van_2021_
kia_k5_sedan_2010_2015
kia_k5_sedan_2020_
kia_k7_sedan_2016_2020
kia_mohave_suv_2020_
kia_morning_hatchback_2004_2010
kia_morning_hatchback_2011_2016
kia_ray_hatchback_2012_2017
kia_sorrento_suv_2015_2019
kia_sorrento_suv_2020_
kia_soul_suv_2014_2018
kia_sportage_suv_2016_2020
kia_stonic_suv_2017_2019
renault_sm3_sedan_2015_2018
renault_xm3_suv_2020_
ssangyong_korando_suv_2019_2020
ssangyong_tivoli_suv_2016_2020

In [ ]:
!cat data/obj.names

In [ ]:
!cp data/obj.names data/coco.names


### data/obj.data

In [ ]:
%%writefile data/obj.data
classes = 34
train = data/train.txt
valid = data/test.txt
names = /content/darknet/data/obj.names
backup = ./

In [ ]:
!cat data/obj.data

In [ ]:
# ls -al data/obj/Raccoon 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/laptops_labeled/*[jpg|png|jpeg]")
images_list = glob.glob("data/obj/Cars/train/*[jpg|png|jpeg]")

# ls -al data/obj 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/*[jpg|png|jpeg]")

print(images_list)

# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!cat data/train.txt

# 4. 사전 학습된 YOLOv3 모델 파일 다운로드

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-06-02 13:15:15--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  70.1MB/s    in 2.2s    

2023-06-02 13:15:18 (70.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 5. 학습관련 설정 파일 수정

원 설정파일 yolov3.cfg를 yolv3_custom.cfg로 복사하고

복사한 설정파일을 수정

In [ ]:
# 탐지 대상 클래스 수를 설정
CLASS_NUM = 34

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

classes = CLASS_NUM
filters = (classes + 5) * 3
# max_batches = classes * 2000
max_batches = classes * 1000
steps1 = int(max_batches*0.8)
steps2 = int(max_batches*0.9)
# if max_batches < 6000:
#   max_batches = 6000

# Edit classes & filters
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = $max_batches/' cfg/yolov3_custom.cfg
!sed -i '610 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '696 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '783 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '603 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i '689 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i '776 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i 's/steps=400000,450000/steps=$steps1,$steps2/' cfg/yolov3_custom.cfg

# 6. 학습 실행

학습된 모델은  ./yolov3_custom_last.weights 로 저장된다.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show 

## 7. 탐지 실행

In [ ]:
!./darknet detect cfg/yolov3_custom.cfg ./yolov3_custom_last.weights data/obj/Cars/valid/raccoon-57

_jpg.rf.26f3fae218f26088f6a7405c58ae8020.jpg


In [ ]:
pred = plt.imread('predictions.jpg')
plt.figure(figsize=(10,10))
plt.imshow(pred)
plt.show()